In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import os
import joblib

In [ ]:
df =  pd.read_csv('')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pos              752 non-null    int64  
 1   name             752 non-null    object 
 2   event_id         752 non-null    int64  
 3   wca_id           752 non-null    object 
 4   average_seconds  752 non-null    float64
 5   ranking_wca      752 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 35.4+ KB


,pos,name,event_id,wca_id,average_seconds,ranking_wca
0,2,Bofan Zhang,333,2021ZHAN01,5.62,8
1,4,Bofan Zhang,333,2021ZHAN01,5.78,8
2,3,Bofan Zhang,333,2021ZHAN01,5.56,8
3,2,Bofan Zhang,333,2021ZHAN01,5.72,8
4,2,Bofan Zhang,333,2021ZHAN01,6.06,8


In [4]:
label_encoder = LabelEncoder()
df['name'] = label_encoder.fit_transform(df['name'])
#label_encoder = LabelEncoder()
#df['wca_id'] = label_encoder.fit_transform(df['wca_id'])

In [8]:
X = df[['average_seconds', 'name', 'pos']]
y = df['en_podio_prob']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)#distribucion de rango 

In [48]:
rfr = RandomForestRegressor(random_state=13)
rfr.fit(X_train, y_train)

RandomForestRegressor(random_state=13)

In [49]:
y_pred = rfr.predict(X_test)

In [50]:
mean_absolute_error(y_pred, y_test)

4.023241073177285e-05

In [51]:
mean_squared_error(y_pred, y_test)

1.1371890110269587e-07

In [52]:
r2_score(y_pred, y_test)

0.9999928512108367

In [81]:
# Agregar columna ranking_wca
#new_data['ranking_wca'] = new_data['name'].map(ranking_wca)
new_data['pos'] = [10, 8, 6, 11, 5, 6, 4, 2, 1]  # ejemplo
# Codificar nombres igual que en el entrenamiento
#new_data['name'] = label_encoder.transform(new_data['name'])

# === 8. Predecir probabilidad de podio ===
X_new = X_new = new_data[['average_seconds', 'name', 'pos']]
new_data['prob_en_podio_predicha'] = rfr.predict(X_new)
new_data['name'] = decoded = label_encoder.inverse_transform(new_data['name'])
# Mostrar resultados
print(new_data[['name', 'average_seconds', 'prob_en_podio_predicha']].sort_values('prob_en_podio_predicha', ascending=False))


                 name  average_seconds  prob_en_podio_predicha
6    Tymon Kolasiński         5.206000                0.996452
3            Max Park         5.647143                0.986301
7         Xuanyi Geng         4.806522                0.983051
8         Yiheng Wang         4.517308                0.977099
1        Luke Garrett         6.252500                0.939024
2  Matty Hiroto Inaba         5.834000                0.882353
5       Teodor Zajder         5.828000                0.807692
4          Ruihang Xu         5.388333                0.645161
0         Bofan Zhang         5.680000                0.519231
